In [2]:
import gefera as gf
import numpy as np
import celerite2 as ct
from numpy.linalg import inv
import matplotlib.pyplot as plt
import math

In [3]:
p_transit = {
'ap':215.0,     #semi-major axis
'tp':-91.25,    #time of periastron passage
'ep':0.0,       #eccentricity
'pp':365,       #orbital period
'wp':0.1 * np.pi / 180,    #argument of periastron
'ip':89.8 * np.pi / 180,   #inclination
'am':2,
'tm': -4.2,
'em' : 0.0,
'pm' : 8,
'om' :90 * np.pi / 180,    #longitude of asending node (in radians)
'wm' : -90 * np.pi / 180,
'im' : 90.0 * np.pi / 180,
'mm' : 0.01,    #moon/planet mass ratio
'rp' : 0.1,
'rm' : 0.05,
'u1' : 0.5,
'u2' : 0.3
}

p_noise = {
    'rho':1,
    'Q':1/np.sqrt(2),
    'sigma':0,
    'diag':0.001
}

po = gf.orbits.PrimaryOrbit(p_transit['ap'], p_transit['tp'], p_transit['ep'], p_transit['pp'], p_transit['wp'], p_transit['ip'])
mo = gf.orbits.SatelliteOrbit(p_transit['am'], p_transit['tm'], p_transit['em'], p_transit['pm'], p_transit['om'], p_transit['wm'], p_transit['im'], p_transit['mm'])
sys = gf.systems.HierarchicalSystem(po, mo)

   
t = np.linspace(-0.6, 0.3, 10000)

flux, grad = sys.lightcurve(t, p_transit['u1'], p_transit['u2'], p_transit['rp'], p_transit['rm'], grad=True)



In [31]:
#reparameterizing omega,t_p to t0
#inspired by Agol https://github.com/ericagol/TRAPPIST1_Spitzer/blob/master/src/NbodyGradient/src/kepler_init.jl
def reparam_t0(e,w,p,tp):
    '''Reparameterizes t0 in terms of e,w,p,tp'''
    e = np.sqrt(e*np.cos(w)**2+e*np.sin(w))
    t_anom = 1.5*np.pi-w
    sqrt1me2 = np.sqrt(1-e**2)
    t0 = tp-p*sqrt1me2*np.pi*e*np.sin(t_anom)/(2*(1+e*np.cos(t_anom)))+2*math.atan2(sqrt1me2*np.tan(0.5*t_anom),1+e)/sqrt1me2
    return t0

In [5]:
def transit_duration(e,P,w,a,i):
    '''Returns transit duration of the planet given:
    e: eccentricity
    P: orbital period
    w: argument of periastron
    a: semi major axis in terms of star radii
    i: inclination
    '''
    f = np.pi/2 - w #true anomaly at center of transit
    S = a*((1-e**2)/(1+e*np.cos(f)))*np.sqrt(1-np.sin(i)**2*np.sin(w+f)**2) #separation between planet and star at center of transit in terms of star radii
    T = (P/np.pi)*(S**2/np.sqrt(1-e**2))*np.arcsin(np.sqrt(1-a**2*S**2*np.cos(i)**2)/(a*S*np.sin(i)))
    return T

In [6]:
def impact_param(r,i):
    b = r*cos(i)
    return b

In [7]:
def phi(t,P):
    phi = 2*np.pi*t/P
    return phi

In [54]:
#derivative of the light curve with respect to t0p and t0m
ep=p_transit['ep']
wp=p_transit['wp']
pp=p_transit['pp']
dt0p_dtp = 1
dt0p_dpp = -np.sqrt(1-ep**2)*np.sin(1.5*np.pi-wp)/(2*(1+ep*np.cos(1.5*np.pi-wp)))
dt0p_dep = 2*ep*np.arctan2(np.sqrt(1-ep**2)*np.tan(0.5*(1.5*np.pi-wp)),(1+ep))/(1-ep**2)**1.5
+ pp*np.sqrt(1-ep**2)*np.cos(1.5*np.pi-wp)*np.sin(1.5*np.pi-wp)/(2*(1+ep*np.cos(1.5*np.pi-wp))**2)
+ ep*pp*np.sin(1.5*np.pi-wp)/(2*np.sqrt(1-ep**2)*(1+ep*np.cos(1.5*np.pi-wp)))
+ 2*(-ep*np.tan(0.5*(1.5*np.pi-wp))/((1+ep)*np.sqrt(1-ep**2))-np.sqrt(1-ep**2)*np.tan(0.5*(1.5*np.pi-wp))/(1+ep)**2)/(np.sqrt(1-ep**2)*(1+(((1-ep**2)*np.tan(0.5*(1.5*np.pi-wp))**2)/(1+ep**2))))
dt0p_dwp = -(np.cos(0.5*(1.5*np.pi-wp))**-1)**2/((1+ep**2)*(1+((1-ep**2)*np.tan(0.5*(1.5*np.pi-wp))**2/(1+ep**2))))

em=p_transit['em']
wm=p_transit['wm']
pm=p_transit['pm']
dt0m_dtm = 1
dt0m_dpm = -np.sqrt(1-em**2)*np.sin(1.5*np.pi-wm)/(2*(1+em*np.cos(1.5*np.pi-wm)))
dt0m_dem = 2*em*np.arctan2(np.sqrt(1-em**2)*np.tan(0.5*(1.5*np.pi-wm)),(1+em))/(1-em**2)**1.5
+ pm*np.sqrt(1-em**2)*np.cos(1.5*np.pi-wm)*np.sin(1.5*np.pi-wm)/(2*(1+em*np.cos(1.5*np.pi-wm))**2)
+ em*pm*np.sin(1.5*np.pi-wm)/(2*np.sqrt(1-em**2)*(1+em*np.cos(1.5*np.pi-wm)))
+ 2*(-em*np.tan(0.5*(1.5*np.pi-wm))/((1+em)*np.sqrt(1-em**2))-np.sqrt(1-em**2)*np.tan(0.5*(1.5*np.pi-wm))/(1+em)**2)/(np.sqrt(1-em**2)*(1+(((1-em**2)*np.tan(0.5*(1.5*np.pi-wm))**2)/(1+em**2))))
dt0p_dwp = -(np.cos(0.5*(1.5*np.pi-wm))**-1)**2/((1+em**2)*(1+((1-em**2)*np.tan(0.5*(1.5*np.pi-wm))**2/(1+em**2))))


In [84]:
len(grad.keys())

18

In [85]:
params = ['r1','r2','a1','a2','u1','u2','p1','p2','e1','e2','i1','i2','o2','m2','t01','t02']
len(params)

16

In [10]:
def cov_matrix(p_transit,p_noise,**ind_var):
    
    param = set(p_transit.keys()).intersection(set(ind_var.keys())) #finds if the ind_var is a transit parameter
    key, values = list(ind_var.items())[0]
    if param != {} and len(param) == 1:
        p_transit.update({key:values})
        update = 0 #used later in the for loop to determine which param should equal val
    elif param == set():
        param = set(p_transit.keys()).intersection(set(ind_var.keys()))
        p_noise.update({key:values})
        update = 1
    else:
        return None #if the ind_var is in neither transit nor noise dictionary, return None
    
    t=np.linspace(-0.6,0.3,10000)
    
    #for each value of the independent variable, create a planet-moon system, instantiate a noise model, and calculate the cov matrix
    for val in values:
        if update == 0:
            p_transit[key] = val
        elif update == 1:
            p_noise[key] = val
    
        po = gf.orbits.PrimaryOrbit(p_transit['ap'], p_transit['tp'], p_transit['ep'], p_transit['pp'], p_transit['wp'], p_transit['ip'])
        mo = gf.orbits.SatelliteOrbit(p_transit['am'], p_transit['tm'], p_transit['em'], p_transit['pm'], p_transit['om'], p_transit['wm'], 
                                      p_transit['im'], p_transit['mm'])
        sys = gf.systems.HierarchicalSystem(po, mo)
        flux, grad = sys.lightcurve(t, p_transit['u1'], p_transit['u2'], p_transit['rp'], p_transit['rm'], grad=True)
        
        #instantiate noise model
        kernel = ct.terms.SHOTerm(rho=p_noise['rho'], Q=p_noise['Q'], sigma=p_noise['sigma'])    
        gp = ct.GaussianProcess(kernel)
        
        #reparameterize to get the transit time of the planet and moon
        t0p = reparam_t0(p_transit['ep'],p_transit['wp'],p_transit['pp'],p_transit['tp'])
        t0m = reparam_t0(p_transit['em'],p_transit['wm'],p_transit['pm'],p_transit['tm'])
        
        #define impact parameters
        b1 = impact_param(p_transit['rp'],p_transit['ip'])
        b2 = impact_param(p_transit['rm'],p_transit['im'])
        
        #define phi
        phi = phi(p_transit['tm'],p_transit['pm'])
                                         
        #define transit duration
        T = transit_duration(p_transit['ep'],p_transit['pp'],p_transit['wp'],p_transit['ap'],p_transit['ip'])
              
        #calculate/assign derivatives of flux with respect to each new param                                
        dt0p = grad['t1']
        dt0m = grad['t2']
        grad['t01'] = dt0p
        grad['t02'] = dt0m
                                         
        dtb1 = (1/np.cos(p_transit['ip']))*grad['r1']-1/(r*np.sin(p_transit['ip']))*grad['ip']
        dtb2 = (1/np.cos(p_transit['im']))*grad['r2']-1/(r*np.sin(p_transit['im']))*grad['im']
        grad['b1'] = dtb1
        grad['b2'] = dtb2
        
        dtphi = p_transit['pm']/(2*np.pi)*grad['t2']-p_transit['pm']**2/(2*np.pi*p_transit['t0m'])*grad['p2']
        grad['phi'] = dtphi
        
        dtT = 1/((1/(np.pi*(1 + p_transit['ep']*np.sin(p_transit['wp']))**2))*p_transit['ap']**2*(1 - p_transit['ep']**2)**(3/2)*
                np.arcsin((np.csc(p_transit['ip'])* (1 + p_transit['ep']* np.sin(p_transit['wp']))*
                np.sqrt(1 - (p_transit['ap']**4*(1 - p_transit['ep']**2)**2*np.cos(p_transit['ip'])**2*(1 - np.sin(p_transit['ip'])**2))/(1 + p_transit['ep']*np.sin(p_transit['wp']))**2))/(
                p_transit['ap']**2*(1 - p_transit['ep']**2)*np.sqrt(1 - np.sin(p_transit['ip'])**2)))*(1 - np.sin(p_transit['ip'])**2)) * grad['p1']
            + 
        grad['T'] = dtT

        
        #params to make up the cov matrix
        params = ['r1','r2','a1','a2','u1','u2','p1','p2','e1','e2','i1','i2','o2','m2','t01','t02']
        
        #calculate and assign entries in info_matrix
        info_matrix = np.zeros((len(params),len(params)))       
        
        #for each valuable parameter, calculate the transpose of its gradient
        for i,u in enumerate(params):
            grad_k = grad[u]
            np_arr = np.array(grad_k)
            transpose_arr = np_arr.transpose()
            
            #calculate each entry of the info matrix
            for j,v in enumerate(params):
                grad_l = grad[v]
                gp.compute(t,diag=p_noise['diag'])
                entry_ij = np.matmul(transpose_arr,grad_l/p_noise['diag'])
                info_matrix[i][j] = entry_ij
        
        cov_matrix = inv(info_matrix)
        
        return cov_matrix
            

In [99]:
def plot_cov(var1,var2,cov_matrices):
    '''This function plots the derivative of var1 with respect
    to var2. It also takes cov_matrices which are calculated from 
    the cov_matrix() function.'''
    
    #i want to pull the one covariance value from each matrix and plot them
    x_vals = np.array([])
    y_vals = np.array([])
    
    

In [102]:
cov_matrix(p_transit,p_noise,rho=[1,1.1,1.2])

array([[ 9.52089294e-03, -6.75551025e-04, -3.89536137e+02,
         1.48477469e+03,  1.61104462e+00, -2.06337981e+00,
        -6.71385422e+02,  1.81002002e+04, -7.61038377e+00,
        -1.52028977e+03, -9.43599346e-03, -5.48925105e-03,
         1.82205746e+00,  4.39962551e+00, -7.16911977e+02,
        -9.05009594e+03],
       [-6.75811371e-04,  1.51805121e-03,  1.62032377e+02,
         7.45785693e+02, -1.87076095e-01,  1.93062202e-01,
         2.75988997e+02,  9.04480522e+03, -3.00495471e+00,
        -7.57712368e+02,  3.41268849e-03, -7.96357937e-03,
        -5.29844091e-01, -1.14024488e+00, -4.17951223e+02,
        -4.52240251e+03],
       [-8.89780673e+02, -8.05785519e+01,  6.80290583e+11,
         4.40855978e+10, -3.73545613e+05,  4.38211877e+05,
         1.15579236e+12,  5.32080793e+11,  1.31272599e+09,
        -4.44675023e+10,  1.10209997e+07, -1.47354093e+04,
         4.70247094e+06,  2.25334628e+07, -1.36111646e+11,
        -2.66040395e+11],
       [ 1.47911651e+03,  7.32456766e

vary white noise, plot as function fo other params
play around with dif combos of valuable params ->>
add params to valuable params, figure out which ones are causing the problem
params that are more directly related to the appearance of the light curve are probably safe
params that matter a lot to the shape of the transit will result in small values in convariance matrix
params that have little influence to the shape will blow up matrix and give weird values

Ideas 11/9:  
I could make the covariance matrix a dictionary with keys that tell me what the entries correspond to so then I can search for two keys in the matrix and make a plot of how they change with the independent variable in cov_matrix.  
Use pandas dataframe to make names for rows and columns, also makes visualization easier.

In [ ]:
#calc dt0
dt0_dtp = 1/(2*())

In [ ]:
def plot_covariance(cov):
    

In [97]:
cov_matrix(p_transit,p_noise,rho=[0.9,1,1.05])

LinAlgError: Singular matrix

good combos: a,r  t,r,u,p,o2,m,i   w,p,m,u,o,i
w,p,t is singular

we want to work with timing of the barycenter transit T instead of w and t
T = timing of the barycenter of the planet-moon system
the time at which the barycenter is lined up w the center of star
pretending the planet-moon mass is just a single planet
assume e=0
figure out long of peri has to be so that t_0 corresponds to the transit of the barycenter

find out which params we are interested in
explore how those params change w dif transits